# README

This notebook allows to prepare dataset of slices.

The dataset is a pickle serialized file containing a dictionary:
* the key is the subject directory name, the value is the set of slices of that subject

All slices are square of a given side, moreover is possible to specify:
* what percentile of the tumor size to use to extract slices
* if slices are tumor crop or full brain

In [1]:
from gliomi import *

Using TensorFlow backend.


In [2]:
class SliceProvider:
    
    def __init__(self, sequence_repo, full_brain=False):
        self.sequence_repo = sequence_repo
        self.full_brain = full_brain
        
    def has(self, subject, sequence_name):
        return self.sequence_repo.has(subject, sequence_name)
        
    """
    Save the slices of the whole brain reshaped with a squared size
    """
    def get_slices(self, sequence_name, subject, side):
        print("Loading slices from:", subject)
        return get_slices_for_subject(self.sequence_repo, 
                                      sequence_name, 
                                      subject, 
                                      side, 
                                      full_brain=self.full_brain)

In [3]:
def load_data_task(item):
    
    gliomi_dataset, subject, sequence_names, side, percentile = item 
    
    print(subject, "started")
    
    images = gliomi_dataset.load_data([subject], sequence_names, side, percentile)
    
    res = subject, normalize(images, max_value=1., axis=(1, 2))
    
    print(subject, "done")
    
    return res


In [4]:
import pickle
import glob
import os
import pandas as pd
from numpy import load
import numpy as np
import sys

import multiprocessing
from multiprocessing import Pool

class GliomiDataset:
    
    def __init__(self, subjects_dir, slice_provider):
        self.slice_provider = slice_provider
        self.subjects_dir = subjects_dir

    """
    Load a set of slices given the ROI sizes and the selected percentile for a single subject.
    Only slices with a ROI size greater or equal that corresponding percentile are considered.
    """
    def load_data_single_channel_for_subject(self, subject, sequence_name, side, percentile):
        
        roi = self.slice_provider.sequence_repo.get_roi(subject)
        
        ((rmin, rmax), (cmin, cmax), (zmin, zmax)) = get_bounding_box(roi)
    
        z_height = zmax - zmin
        width = rmax - rmin
        height = cmax - cmin
    
        resampled_roi = mask_crop_resize(roi, roi, width, height, z_height)

        roi_sizes = get_roi_size(resampled_roi, 2)

        # Load slices
        slices = self.slice_provider.get_slices(sequence_name, subject, side)
        
        assert slices.shape[2] == roi_sizes.shape[0]

        print(f"Loaded {subject}: {slices.shape}, {roi_sizes.shape}")

        return slices[:,:,ordered_index_percentile_of_sizes(roi_sizes, percentile)]    
    
    """
    Load a set of slices given the ROI sizes and the selected percentile for a set of subjects.
    Only slices with a ROI size greater or equal that corresponding percentile are considered.
    Images channels correspondes to specified sequences given in the same order.
    """
    def load_data(self, subjects, sequence_names, side, percentile):
        slices = [
            np.stack([
                self.load_data_single_channel_for_subject(subject, sequence_name, side, percentile) 
                for sequence_name in sequence_names], axis=3)
            for subject in subjects
        ]
        return np.moveaxis(np.concatenate(slices, axis=2), 2, 0)
    
    """
    Return a valid list of subjects that has all the sequences specified.
    Subjects are selected between the list of specified
    """
    def get_valid_subject_list(self, subjects, sequence_names):
        ret = []
        for subject in subjects:
            admit = True
            for sequence_name in sequence_names:
                if not self.slice_provider.has(subject, sequence_name):
                    admit = False
            if admit:
                ret.append(subject)
        return ret
    
    """
    Return list of subjects in a dataset dir: all the directory names are returned as subjects
    """
    def get_subject_list(self):
        ret = []
        for file in glob.glob(f"{self.subjects_dir}/*"):
            if os.path.isdir(file):
                ret.append(os.path.basename(file))
        return ret
    

    """
    Given subject path and a table of subjects and labels for a given side
    Create a structure that hosts all related slices
    """
    def load_dataset(self, sequence_names, side, percentile):

        all_subjects = self.get_subject_list()

        # Return only subjects that has all sequences
        valid_subjects = self.get_valid_subject_list(
            all_subjects, 
            sequence_names)

        X = {}
        tasks = []
        for i, subject in enumerate(valid_subjects):
            # tasks = load_data_task((self, subject, sequence_names, side, percentile))
            tasks.append((self, subject, sequence_names, side, percentile))

        # Parallel processing
        processing_pool = Pool(int(multiprocessing.cpu_count()))
        result = processing_pool.map(load_data_task, tasks)

        for subject, slices in result:
            X[subject] = slices

        return X

# Generation of Bratz dataset

In [5]:
%%time

side = 64

percentile = 100

sequence_repo = SequenceRepoBrats19("/data/RMN2/datasets/BRATS2019/HGG_Training_data_Brats_2019")

slice_provider = SliceProvider(sequence_repo, full_brain=True)

dataset = GliomiDataset("/data/RMN2/datasets/BRATS2019/HGG_Training_data_Brats_2019/scaled", slice_provider)

for sequence_name in ["t1","t2","t1ce","flair"]:

    X = dataset.load_dataset([sequence_name], side, percentile)

    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-full-brain/bratz-{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)

BraTS19_CBICA_AUN_1 started
BraTS19_TCIA08_218_1 started
BraTS19_CBICA_AWH_1 started
BraTS19_2013_17_1 started
BraTS19_CBICA_AXQ_1 started
BraTS19_TCIA06_184_1 started
BraTS19_CBICA_AQU_1 started
BraTS19_CBICA_BJY_1 started
BraTS19_TCIA04_328_1 started
BraTS19_CBICA_ASK_1 started
BraTS19_TCIA02_606_1 started
BraTS19_TCIA03_498_1 started
BraTS19_CBICA_ATN_1 started
BraTS19_CBICA_ARF_1 started
BraTS19_CBICA_AOO_1 started
BraTS19_CBICA_ATV_1 started
BraTS19_CBICA_ASF_1 started
BraTS19_CBICA_AVJ_1 started
BraTS19_CBICA_ANZ_1 started
BraTS19_TCIA01_231_1 started
BraTS19_TCIA01_460_1 started
BraTS19_TCIA01_131_1 started
BraTS19_TCIA06_372_1 started
BraTS19_TCIA01_378_1 started
BraTS19_TCIA02_309_1 started
BraTS19_CBICA_AQO_1 started
BraTS19_2013_25_1 started
BraTS19_2013_2_1 started
BraTS19_2013_21_1 started
BraTS19_TCIA04_192_1 started
BraTS19_TCIA01_150_1 started
BraTS19_CBICA_AXW_1 started
BraTS19_TCIA08_242_1 started
BraTS19_TCIA02_135_1 started
BraTS19_CBICA_AZD_1 started
BraTS19_TCIA06

Loading slices from: BraTS19_CBICA_BEM_1
Loading slices from: BraTS19_2013_4_1
Loading slices from: BraTS19_CBICA_AUA_1
Loading slices from: BraTS19_2013_23_1
Loading slices from: BraTS19_CBICA_AVT_1
Loading slices from: BraTS19_CBICA_BKV_1
Loading slices from: BraTS19_2013_18_1
Loaded BraTS19_CBICA_ARW_1: (64, 64, 55), (55,)
BraTS19_CBICA_ARW_1 done
BraTS19_TCIA02_368_1 started
Loaded BraTS19_TCIA03_121_1: (64, 64, 59), (59,)
BraTS19_TCIA03_121_1 done
BraTS19_TCIA02_208_1 started
Loaded BraTS19_CBICA_AQZ_1: (64, 64, 82), (82,)
BraTS19_CBICA_AQZ_1 done
BraTS19_CBICA_BIC_1 started
Loaded BraTS19_TCIA02_179_1: (64, 64, 69), (69,)
BraTS19_TCIA02_179_1 done
BraTS19_TCIA01_235_1 started
Loaded BraTS19_CBICA_AYI_1: (64, 64, 28), (28,)
BraTS19_CBICA_AYI_1 done
BraTS19_TCIA02_394_1 started
Loaded BraTS19_TCIA02_314_1: (64, 64, 88), (88,)
BraTS19_TCIA02_314_1 done
BraTS19_CBICA_AWV_1 started
Loaded BraTS19_TCIA03_375_1: (64, 64, 57), (57,)
BraTS19_TCIA03_375_1 done
BraTS19_TCIA01_401_1 started


BraTS19_CBICA_ASG_1 done
BraTS19_TCIA05_396_1 started
Loaded BraTS19_TCIA03_199_1: (64, 64, 81), (81,)
BraTS19_TCIA03_199_1 done
BraTS19_CBICA_ATX_1 started
Loaded BraTS19_CBICA_AQQ_1: (64, 64, 62), (62,)
BraTS19_CBICA_AQQ_1 done
BraTS19_CBICA_AAB_1 started
Loaded BraTS19_TCIA01_201_1: (64, 64, 73), (73,)
BraTS19_TCIA01_201_1 done
BraTS19_TCIA08_113_1 started
Loaded BraTS19_TCIA06_211_1: (64, 64, 76), (76,)
BraTS19_TCIA06_211_1 done
BraTS19_TCIA02_370_1 started
Loaded BraTS19_CBICA_AWV_1: (64, 64, 42), (42,)
BraTS19_CBICA_AWV_1 done
BraTS19_CBICA_ARZ_1 started
Loaded BraTS19_TCIA06_603_1: (64, 64, 80), (80,)
BraTS19_TCIA06_603_1 done
BraTS19_CBICA_APK_1 started
Loaded BraTS19_CBICA_AQT_1: (64, 64, 70), (70,)
BraTS19_CBICA_AQT_1 done
BraTS19_2013_5_1 started
Loaded BraTS19_CBICA_BIC_1: (64, 64, 66), (66,)
BraTS19_CBICA_BIC_1 done
BraTS19_TCIA02_117_1 started
Loaded BraTS19_CBICA_AOD_1: (64, 64, 79), (79,)
BraTS19_CBICA_AOD_1 done
BraTS19_CBICA_BGO_1 started
Loaded BraTS19_TMC_06290_1: (

Loading slices from: BraTS19_CBICA_ASY_1
Loading slices from: BraTS19_TCIA01_335_1
Loading slices from: BraTS19_CBICA_AUQ_1
Loading slices from: BraTS19_2013_20_1
Loading slices from: BraTS19_TCIA01_429_1
Loading slices from: BraTS19_CBICA_AOH_1
Loading slices from: BraTS19_CBICA_AZH_1
Loading slices from: BraTS19_CBICA_APR_1
Loading slices from: BraTS19_TCIA04_111_1
Loading slices from: BraTS19_CBICA_AYA_1
Loading slices from: BraTS19_TCIA03_419_1
Loading slices from: BraTS19_TCIA02_222_1
Loading slices from: BraTS19_CBICA_AOC_1
Loading slices from: BraTS19_CBICA_ANP_1
Loading slices from: BraTS19_TCIA08_167_1
Loading slices from: BraTS19_CBICA_AQR_1
Loading slices from: BraTS19_TMC_12866_1
Loading slices from: BraTS19_TCIA02_455_1
Loading slices from: BraTS19_CBICA_ABO_1
Loading slices from: BraTS19_TCIA04_437_1
Loading slices from: BraTS19_CBICA_ANG_1
Loading slices from: BraTS19_2013_7_1
Loading slices from: BraTS19_2013_10_1
Loading slices from: BraTS19_CBICA_BGT_1
Loading slices 

Loaded BraTS19_CBICA_BHQ_1: (64, 64, 33), (33,)
BraTS19_CBICA_BHQ_1 done
Loaded BraTS19_TCIA01_190_1: (64, 64, 44), (44,)
BraTS19_TCIA01_190_1 done
Loaded BraTS19_TCIA01_186_1: (64, 64, 51), (51,)
BraTS19_TCIA01_186_1 done
Loaded BraTS19_TCIA01_147_1: (64, 64, 94), (94,)
BraTS19_TCIA01_147_1 done
Loaded BraTS19_TCIA02_473_1: (64, 64, 50), (50,)
BraTS19_TCIA02_473_1 done
Loaded BraTS19_TCIA02_605_1: (64, 64, 57), (57,)
BraTS19_TCIA02_605_1 done
Loaded BraTS19_TCIA08_278_1: (64, 64, 55), (55,)
BraTS19_TCIA08_278_1 done
Loaded BraTS19_CBICA_AQN_1: (64, 64, 46), (46,)
BraTS19_CBICA_AQN_1 done
Loaded BraTS19_CBICA_ALN_1: (64, 64, 72), (72,)
BraTS19_CBICA_ALN_1 done
Loaded BraTS19_CBICA_BLJ_1: (64, 64, 48), (48,)
BraTS19_CBICA_BLJ_1 done
Loaded BraTS19_CBICA_AXJ_1: (64, 64, 81), (81,)
BraTS19_CBICA_AXJ_1 done
Loaded BraTS19_CBICA_ASV_1: (64, 64, 80), (80,)
BraTS19_CBICA_ASV_1 done
Loaded BraTS19_CBICA_ABM_1: (64, 64, 53), (53,)
BraTS19_CBICA_ABM_1 done
Loaded BraTS19_TMC_15477_1: (64, 64, 59

...

BraTS19_CBICA_ABY_1 started
Loaded BraTS19_TCIA01_131_1: (64, 64, 76), (76,)
BraTS19_TCIA01_131_1 done
BraTS19_2013_18_1 started
Loaded BraTS19_CBICA_ANZ_1: (64, 64, 48), (48,)
BraTS19_CBICA_ANZ_1 done
BraTS19_TCIA08_162_1 started
Loaded BraTS19_TCIA08_242_1: (64, 64, 77), (77,)
BraTS19_TCIA08_242_1 done
BraTS19_CBICA_BFB_1 started
Loaded BraTS19_CBICA_ATV_1: (64, 64, 59), (59,)
BraTS19_CBICA_ATV_1 done
BraTS19_CBICA_AYI_1 started
Loaded BraTS19_TCIA02_309_1: (64, 64, 81), (81,)
BraTS19_TCIA02_309_1 done
BraTS19_TCIA03_375_1 started
Loaded BraTS19_TCIA04_192_1: (64, 64, 66), (66,)
BraTS19_TCIA04_192_1 done
BraTS19_TCIA08_205_1 started
Loaded BraTS19_CBICA_AXO_1: (64, 64, 93), (93,)
BraTS19_CBICA_AXO_1 done
BraTS19_TCIA01_390_1 started
Loaded BraTS19_TCIA08_218_1: (64, 64, 55), (55,)
BraTS19_TCIA08_218_1 done
BraTS19_CBICA_BBG_1 started
Loaded BraTS19_CBICA_ASK_1: (64, 64, 75), (75,)
BraTS19_CBICA_ASK_1 done
BraTS19_CBICA_BKV_1 started
Loaded BraTS19_CBICA_AUN_1: (64, 64, 81), (81,)
Bra

BraTS19_2013_23_1 done
BraTS19_CBICA_AVF_1 started
Loaded BraTS19_TCIA05_277_1: (64, 64, 93), (93,)
BraTS19_TCIA05_277_1 done
BraTS19_TCIA05_444_1 started
Loaded BraTS19_2013_18_1: (64, 64, 81), (81,)
BraTS19_2013_18_1 done
BraTS19_CBICA_AQA_1 started
Loaded BraTS19_CBICA_BAN_1: (64, 64, 96), (96,)
BraTS19_CBICA_BAN_1 done
BraTS19_TCIA06_211_1 started
Loading slices from: BraTS19_TCIA02_208_1
Loading slices from: BraTS19_TCIA01_235_1
Loading slices from: BraTS19_TCIA02_368_1
Loading slices from: BraTS19_TCIA08_319_1
Loading slices from: BraTS19_TCIA01_201_1
Loading slices from: BraTS19_TCIA02_394_1
Loading slices from: BraTS19_TCIA01_401_1
Loading slices from: BraTS19_CBICA_AAP_1
Loading slices from: BraTS19_CBICA_ALU_1
Loading slices from: BraTS19_TCIA06_603_1
Loading slices from: BraTS19_CBICA_BIC_1
Loading slices from: BraTS19_TCIA01_203_1
Loading slices from: BraTS19_CBICA_AWV_1
Loading slices from: BraTS19_CBICA_BGX_1
Loading slices from: BraTS19_CBICA_ASW_1
Loading slices from: B

BraTS19_CBICA_ASH_1 started
Loaded BraTS19_TCIA08_113_1: (64, 64, 67), (67,)
BraTS19_TCIA08_113_1 done
BraTS19_TCIA04_149_1 started
Loaded BraTS19_CBICA_ARZ_1: (64, 64, 64), (64,)
BraTS19_CBICA_ARZ_1 done
BraTS19_TCIA02_151_1 started
Loaded BraTS19_TCIA03_133_1: (64, 64, 52), (52,)
BraTS19_TCIA03_133_1 done
BraTS19_CBICA_BCF_1 started
Loaded BraTS19_TCIA02_117_1: (64, 64, 73), (73,)
BraTS19_TCIA02_117_1 done
BraTS19_CBICA_AME_1 started
Loaded BraTS19_TCIA02_377_1: (64, 64, 71), (71,)
BraTS19_TCIA02_377_1 done
BraTS19_CBICA_AUX_1 started
Loaded BraTS19_TCIA02_283_1: (64, 64, 77), (77,)
BraTS19_TCIA02_283_1 done
BraTS19_CBICA_AMH_1 started
Loaded BraTS19_CBICA_AOS_1: (64, 64, 69), (69,)
BraTS19_CBICA_AOS_1 done
BraTS19_TCIA02_168_1 started
Loaded BraTS19_CBICA_AAB_1: (64, 64, 82), (82,)
BraTS19_CBICA_AAB_1 done
BraTS19_CBICA_APY_1 started
Loaded BraTS19_TCIA02_607_1: (64, 64, 48), (48,)
BraTS19_TCIA02_607_1 done
BraTS19_TCIA01_335_1 started
Loaded BraTS19_TCIA05_396_1: (64, 64, 75), (75,

Loading slices from: BraTS19_CBICA_BHQ_1
Loading slices from: BraTS19_TCIA01_147_1
Loaded BraTS19_TCIA03_419_1: (64, 64, 71), (71,)
BraTS19_TCIA03_419_1 done
BraTS19_CBICA_AQG_1 started
Loaded BraTS19_2013_7_1: (64, 64, 65), (65,)
BraTS19_2013_7_1 done
BraTS19_TCIA03_257_1 started
Loaded BraTS19_TMC_12866_1: (64, 64, 85), (85,)
BraTS19_TMC_12866_1 done
BraTS19_2013_14_1 started
Loaded BraTS19_CBICA_AOC_1: (64, 64, 44), (44,)
BraTS19_CBICA_AOC_1 done
BraTS19_CBICA_BAX_1 started
Loaded BraTS19_TCIA02_222_1: (64, 64, 85), (85,)
BraTS19_TCIA02_222_1 done
BraTS19_CBICA_ABM_1 started
Loaded BraTS19_CBICA_AOH_1: (64, 64, 50), (50,)
BraTS19_CBICA_AOH_1 done
BraTS19_TCIA02_374_1 started
Loaded BraTS19_CBICA_ANG_1: (64, 64, 79), (79,)
BraTS19_CBICA_ANG_1 done
BraTS19_TCIA03_338_1 started
Loaded BraTS19_TCIA04_111_1: (64, 64, 60), (60,)
BraTS19_TCIA04_111_1 done
BraTS19_CBICA_BHM_1 started
Loaded BraTS19_TCIA08_167_1: (64, 64, 84), (84,)
BraTS19_TCIA08_167_1 done
BraTS19_CBICA_AVB_1 started
Loadi

BraTS19_2013_12_1 done
Loaded BraTS19_CBICA_AYC_1: (64, 64, 79), (79,)
BraTS19_CBICA_AYC_1 done
Loaded BraTS19_CBICA_BHZ_1: (64, 64, 68), (68,)
BraTS19_CBICA_BHZ_1 done


In [8]:
%%time

side = 64

percentile = 100

sequence_repo = SequenceRepoBrats19("/data/RMN2/datasets/BRATS2019/HGG_Training_data_Brats_2019")

slice_provider = SliceProvider(sequence_repo, full_brain=False)

dataset = GliomiDataset("/data/RMN2/datasets/BRATS2019/HGG_Training_data_Brats_2019/scaled", slice_provider)

for sequence_name in ["t1","t2","t1ce","flair"]:

    X = dataset.load_dataset([sequence_name], side, percentile)

    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-tumor-crop/bratz-{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)

BraTS19_CBICA_AUN_1 started
BraTS19_CBICA_ATV_1 started
BraTS19_2013_25_1 started
BraTS19_TCIA02_135_1 started
BraTS19_CBICA_AOO_1 started
BraTS19_TCIA02_309_1 started
BraTS19_CBICA_AXQ_1 started
BraTS19_2013_17_1 started
BraTS19_CBICA_ASF_1 started
BraTS19_2013_21_1 started
BraTS19_TCIA04_328_1 started
BraTS19_CBICA_ATN_1 started
BraTS19_CBICA_AZD_1 started
BraTS19_CBICA_AVJ_1 started
BraTS19_CBICA_AQU_1 started
BraTS19_TCIA04_192_1 started
BraTS19_TCIA06_184_1 started
BraTS19_TCIA02_491_1 started
BraTS19_TCIA01_131_1 started
BraTS19_TCIA08_218_1 started
BraTS19_2013_2_1 started
BraTS19_CBICA_AXO_1 started
BraTS19_CBICA_ARF_1 started
BraTS19_TCIA01_460_1 started
BraTS19_TCIA01_378_1 started
BraTS19_CBICA_ANI_1 started
BraTS19_TCIA01_231_1 started
BraTS19_CBICA_AQO_1 started
BraTS19_TCIA06_372_1 started
BraTS19_TCIA02_300_1 started
BraTS19_TCIA06_409_1 started
BraTS19_TCIA08_242_1 started
BraTS19_TCIA01_150_1 started
BraTS19_CBICA_AXW_1 started
BraTS19_CBICA_ASK_1 started
BraTS19_TCIA0

Loading slices from: BraTS19_CBICA_AOZ_1
Loading slices from: BraTS19_CBICA_AVT_1
Loading slices from: BraTS19_CBICA_BEM_1
Loading slices from: BraTS19_CBICA_AUA_1
Loading slices from: BraTS19_2013_11_1
Loading slices from: BraTS19_TCIA02_314_1
Loading slices from: BraTS19_2013_23_1
Loaded BraTS19_CBICA_AYI_1: (64, 64, 28), (28,)
BraTS19_CBICA_AYI_1 done
BraTS19_TCIA02_368_1 started
Loaded BraTS19_TCIA02_198_1: (64, 64, 54), (54,)
BraTS19_TCIA02_198_1 done
BraTS19_TCIA02_208_1 started
Loaded BraTS19_CBICA_AWG_1: (64, 64, 49), (49,)
BraTS19_CBICA_AWG_1 done
BraTS19_CBICA_BIC_1 started
Loaded BraTS19_CBICA_AAL_1: (64, 64, 51), (51,)
BraTS19_CBICA_AAL_1 done
BraTS19_TCIA01_235_1 started
Loaded BraTS19_TCIA02_322_1: (64, 64, 35), (35,)
BraTS19_TCIA02_322_1 done
BraTS19_TCIA02_394_1 started
Loaded BraTS19_CBICA_AVG_1: (64, 64, 39), (39,)
BraTS19_CBICA_AVG_1 done
BraTS19_CBICA_AWV_1 started
Loaded BraTS19_CBICA_ARW_1: (64, 64, 55), (55,)
BraTS19_CBICA_ARW_1 done
BraTS19_TCIA01_401_1 started


BraTS19_CBICA_BGX_1 done
BraTS19_CBICA_BAP_1 started
Loaded BraTS19_CBICA_ASO_1: (64, 64, 85), (85,)
BraTS19_CBICA_ASO_1 done
BraTS19_TCIA01_425_1 started
Loaded BraTS19_CBICA_AQT_1: (64, 64, 70), (70,)
BraTS19_CBICA_AQT_1 done
BraTS19_2013_5_1 started
Loaded BraTS19_TCIA06_332_1: (64, 64, 78), (78,)
BraTS19_TCIA06_332_1 done
BraTS19_TCIA02_607_1 started
Loaded BraTS19_CBICA_AAG_1: (64, 64, 54), (54,)
BraTS19_CBICA_AAG_1 done
BraTS19_TCIA02_118_1 started
Loaded BraTS19_TCIA03_199_1: (64, 64, 81), (81,)
BraTS19_TCIA03_199_1 done
BraTS19_CBICA_ATX_1 started
Loaded BraTS19_CBICA_BGR_1: (64, 64, 58), (58,)
BraTS19_CBICA_BGR_1 done
BraTS19_CBICA_ANV_1 started
Loaded BraTS19_TCIA01_401_1: (64, 64, 91), (91,)
BraTS19_TCIA01_401_1 done
BraTS19_CBICA_AOS_1 started
Loaded BraTS19_TCIA04_343_1: (64, 64, 56), (56,)
BraTS19_TCIA04_343_1 done
BraTS19_CBICA_ASR_1 started
Loaded BraTS19_TMC_06290_1: (64, 64, 77), (77,)
BraTS19_TMC_06290_1 done
BraTS19_CBICA_AVV_1 started
Loaded BraTS19_CBICA_AVF_1: (6

Loading slices from: BraTS19_CBICA_BCF_1
Loading slices from: BraTS19_TCIA02_222_1
Loading slices from: BraTS19_CBICA_ASY_1
Loading slices from: BraTS19_CBICA_AUX_1
Loading slices from: BraTS19_CBICA_AYA_1
Loading slices from: BraTS19_CBICA_ANP_1
Loading slices from: BraTS19_CBICA_AOH_1
Loading slices from: BraTS19_TCIA03_419_1
Loading slices from: BraTS19_TCIA04_437_1
Loading slices from: BraTS19_TCIA04_111_1
Loading slices from: BraTS19_CBICA_AZH_1
Loading slices from: BraTS19_TCIA08_167_1
Loading slices from: BraTS19_CBICA_ANG_1
Loading slices from: BraTS19_CBICA_AUQ_1
Loading slices from: BraTS19_CBICA_APR_1
Loading slices from: BraTS19_TCIA02_455_1
Loading slices from: BraTS19_CBICA_AOC_1
Loading slices from: BraTS19_TMC_12866_1
Loading slices from: BraTS19_2013_7_1
Loading slices from: BraTS19_CBICA_BGT_1
Loading slices from: BraTS19_CBICA_ABO_1
Loading slices from: BraTS19_2013_10_1
Loading slices from: BraTS19_2013_20_1
Loading slices from: BraTS19_TCIA01_429_1
Loading slices f

Loading slices from: BraTS19_CBICA_AOP_1
Loaded BraTS19_CBICA_AQN_1: (64, 64, 46), (46,)
BraTS19_CBICA_AQN_1 done
Loaded BraTS19_CBICA_ABM_1: (64, 64, 53), (53,)
BraTS19_CBICA_ABM_1 done
Loaded BraTS19_TCIA08_278_1: (64, 64, 55), (55,)
BraTS19_TCIA08_278_1 done
Loaded BraTS19_2013_3_1: (64, 64, 59), (59,)
BraTS19_2013_3_1 done
Loaded BraTS19_CBICA_AVB_1: (64, 64, 22), (22,)
BraTS19_CBICA_AVB_1 done
Loaded BraTS19_CBICA_BHM_1: (64, 64, 66), (66,)
BraTS19_CBICA_BHM_1 done
Loaded BraTS19_TCIA08_234_1: (64, 64, 68), (68,)
BraTS19_TCIA08_234_1 done
Loaded BraTS19_TCIA02_430_1: (64, 64, 63), (63,)
BraTS19_TCIA02_430_1 done
Loaded BraTS19_TMC_15477_1: (64, 64, 59), (59,)
BraTS19_TMC_15477_1 done
Loaded BraTS19_CBICA_ASN_1: (64, 64, 71), (71,)
BraTS19_CBICA_ASN_1 done
Loaded BraTS19_TCIA02_608_1: (64, 64, 71), (71,)
BraTS19_TCIA02_608_1 done
Loaded BraTS19_CBICA_ATB_1: (64, 64, 71), (71,)
BraTS19_CBICA_ATB_1 done
Loaded BraTS19_CBICA_ALN_1: (64, 64, 72), (72,)
BraTS19_CBICA_ALN_1 done
Loaded B

...

BraTS19_TCIA02_322_1 started
Loaded BraTS19_TCIA02_606_1: (64, 64, 73), (73,)
BraTS19_TCIA02_606_1 done
BraTS19_CBICA_BEM_1 started
Loaded BraTS19_CBICA_ASK_1: (64, 64, 75), (75,)
BraTS19_CBICA_ASK_1 done
BraTS19_CBICA_BKV_1 started
Loaded BraTS19_TCIA02_135_1: (64, 64, 60), (60,)
BraTS19_TCIA02_135_1 done
BraTS19_TCIA08_105_1 started
Loaded BraTS19_CBICA_BJY_1: (64, 64, 58), (58,)
BraTS19_CBICA_BJY_1 done
BraTS19_TCIA02_179_1 started
Loaded BraTS19_TCIA01_150_1: (64, 64, 54), (54,)
BraTS19_TCIA01_150_1 done
BraTS19_CBICA_ABY_1 started
Loaded BraTS19_TCIA06_409_1: (64, 64, 71), (71,)
BraTS19_TCIA06_409_1 done
BraTS19_CBICA_ASU_1 started
Loaded BraTS19_TCIA01_131_1: (64, 64, 76), (76,)
BraTS19_TCIA01_131_1 done
BraTS19_2013_18_1 started
Loaded BraTS19_TCIA03_498_1: (64, 64, 62), (62,)
BraTS19_TCIA03_498_1 done
BraTS19_2013_4_1 started
Loaded BraTS19_CBICA_AQO_1: (64, 64, 67), (67,)
BraTS19_CBICA_AQO_1 done
BraTS19_CBICA_AWG_1 started
Loaded BraTS19_CBICA_ARF_1: (64, 64, 67), (67,)
BraTS

Loaded BraTS19_2013_18_1: (64, 64, 81), (81,)
BraTS19_2013_18_1 done
BraTS19_TCIA04_343_1 started
Loading slices from: BraTS19_TCIA02_368_1
Loaded BraTS19_TCIA01_390_1: (64, 64, 74), (74,)
BraTS19_TCIA01_390_1 done
BraTS19_CBICA_AVF_1 started
Loaded BraTS19_CBICA_BBG_1: (64, 64, 80), (80,)
BraTS19_CBICA_BBG_1 done
BraTS19_TCIA05_444_1 started
Loaded BraTS19_2013_11_1: (64, 64, 75), (75,)
BraTS19_2013_11_1 done
BraTS19_CBICA_AQA_1 started
Loaded BraTS19_2013_23_1: (64, 64, 71), (71,)
BraTS19_2013_23_1 done
BraTS19_TCIA06_211_1 started
Loading slices from: BraTS19_TCIA02_208_1
Loading slices from: BraTS19_TCIA02_394_1
Loading slices from: BraTS19_TCIA01_235_1
Loading slices from: BraTS19_TCIA01_401_1
Loading slices from: BraTS19_CBICA_AWV_1
Loading slices from: BraTS19_CBICA_BIC_1
Loading slices from: BraTS19_CBICA_ASW_1
Loading slices from: BraTS19_CBICA_ALU_1
Loading slices from: BraTS19_TCIA01_201_1
Loading slices from: BraTS19_CBICA_AAP_1
Loading slices from: BraTS19_CBICA_BGX_1
Load

BraTS19_CBICA_ARZ_1 done
BraTS19_2013_19_1 started
Loaded BraTS19_CBICA_ABN_1: (64, 64, 51), (51,)
BraTS19_CBICA_ABN_1 done
BraTS19_CBICA_ASH_1 started
Loaded BraTS19_CBICA_BHK_1: (64, 64, 41), (41,)
BraTS19_CBICA_BHK_1 done
BraTS19_TCIA04_149_1 started
Loaded BraTS19_TCIA08_280_1: (64, 64, 48), (48,)
BraTS19_TCIA08_280_1 done
BraTS19_TCIA02_151_1 started
Loaded BraTS19_TCIA02_377_1: (64, 64, 71), (71,)
BraTS19_TCIA02_377_1 done
BraTS19_CBICA_BCF_1 started
Loaded BraTS19_CBICA_AXM_1: (64, 64, 50), (50,)
Loaded BraTS19_CBICA_AQD_1: (64, 64, 64), (64,)
BraTS19_CBICA_AQD_1 done
BraTS19_CBICA_AXM_1 done
BraTS19_CBICA_AME_1 started
BraTS19_CBICA_AUX_1 started
Loaded BraTS19_TCIA01_448_1: (64, 64, 59), (59,)
BraTS19_TCIA01_448_1 done
BraTS19_CBICA_AMH_1 started
Loaded BraTS19_CBICA_ATP_1: (64, 64, 53), (53,)
BraTS19_CBICA_ATP_1 done
BraTS19_TCIA02_168_1 started
Loaded BraTS19_TCIA01_425_1: (64, 64, 47), (47,)
BraTS19_TCIA01_425_1 done
BraTS19_CBICA_APY_1 started
Loaded BraTS19_TCIA02_117_1: 

Loaded BraTS19_TCIA01_429_1: (64, 64, 80), (80,)
BraTS19_TCIA01_429_1 done
BraTS19_TMC_11964_1 started
Loaded BraTS19_CBICA_ANG_1: (64, 64, 79), (79,)
BraTS19_CBICA_ANG_1 done
BraTS19_TCIA03_338_1 started
Loaded BraTS19_2013_7_1: (64, 64, 65), (65,)
BraTS19_2013_7_1 done
BraTS19_TCIA03_257_1 started
Loaded BraTS19_CBICA_AUX_1: (64, 64, 65), (65,)
BraTS19_CBICA_AUX_1 done
BraTS19_CBICA_AXJ_1 started
Loaded BraTS19_TCIA08_167_1: (64, 64, 84), (84,)
BraTS19_TCIA08_167_1 done
BraTS19_CBICA_AVB_1 started
Loaded BraTS19_TCIA02_222_1: (64, 64, 85), (85,)
BraTS19_TCIA02_222_1 done
BraTS19_CBICA_ABM_1 started
Loaded BraTS19_2013_10_1: (64, 64, 73), (73,)
BraTS19_2013_10_1 done
BraTS19_TCIA02_290_1 started
Loaded BraTS19_CBICA_ABO_1: (64, 64, 80), (80,)
BraTS19_CBICA_ABO_1 done
BraTS19_TCIA08_278_1 started
Loaded BraTS19_CBICA_BCF_1: (64, 64, 90), (90,)
BraTS19_CBICA_BCF_1 done
BraTS19_TCIA01_190_1 started
Loaded BraTS19_2013_20_1: (64, 64, 83), (83,)
BraTS19_2013_20_1 done
BraTS19_CBICA_AQN_1 s

Loaded BraTS19_2013_12_1: (64, 64, 73), (73,)
BraTS19_2013_12_1 done
Loaded BraTS19_CBICA_BGN_1: (64, 64, 70), (70,)
BraTS19_CBICA_BGN_1 done
Loaded BraTS19_CBICA_AYC_1: (64, 64, 79), (79,)
BraTS19_CBICA_AYC_1 done
Loaded BraTS19_CBICA_BHZ_1: (64, 64, 68), (68,)
BraTS19_CBICA_BHZ_1 done
CPU times: user 29.2 s, sys: 58.1 s, total: 1min 27s
Wall time: 20min 11s


In [9]:
%%time

side = 128

percentile = 100

sequence_repo = SequenceRepoBrats19("/data/RMN2/datasets/BRATS2019/HGG_Training_data_Brats_2019")

slice_provider = SliceProvider(sequence_repo, full_brain=True)

dataset = GliomiDataset("/data/RMN2/datasets/BRATS2019/HGG_Training_data_Brats_2019/scaled", slice_provider)

for sequence_name in ["t1","t2","t1ce","flair"]:

    X = dataset.load_dataset([sequence_name], side, percentile)

    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-full-brain/bratz-{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)

BraTS19_CBICA_AXQ_1 started
BraTS19_CBICA_BJY_1 started
BraTS19_2013_17_1 started
BraTS19_CBICA_AVJ_1 started
BraTS19_TCIA08_218_1 started
BraTS19_CBICA_AQU_1 started
BraTS19_CBICA_AUN_1 started
BraTS19_TCIA06_184_1 started
BraTS19_CBICA_ATV_1 started
BraTS19_CBICA_AWH_1 started
BraTS19_TCIA01_231_1 started
BraTS19_CBICA_AOO_1 started
BraTS19_TCIA04_328_1 started
BraTS19_CBICA_ANZ_1 started
BraTS19_TCIA02_606_1 started
BraTS19_TCIA01_460_1 started
BraTS19_CBICA_ASK_1 started
BraTS19_CBICA_AQO_1 started
BraTS19_CBICA_ASF_1 started
BraTS19_2013_21_1 started
BraTS19_TCIA06_372_1 started
BraTS19_CBICA_ATN_1 started
BraTS19_TCIA03_498_1 started
BraTS19_TCIA02_491_1 started
BraTS19_TCIA01_131_1 started
BraTS19_CBICA_ARF_1 started
BraTS19_TCIA04_192_1 started
BraTS19_TCIA02_135_1 started
BraTS19_TCIA02_309_1 started
BraTS19_2013_2_1 started
BraTS19_2013_25_1 started
BraTS19_CBICA_AZD_1 started
BraTS19_TCIA08_242_1 started
BraTS19_CBICA_AXW_1 started
BraTS19_TCIA06_409_1 started
BraTS19_CBICA_

Loading slices from: BraTS19_TCIA08_205_1
Loading slices from: BraTS19_CBICA_AOZ_1
Loading slices from: BraTS19_2013_11_1
Loading slices from: BraTS19_TCIA02_274_1
Loading slices from: BraTS19_CBICA_BBG_1
Loading slices from: BraTS19_CBICA_AYI_1
Loading slices from: BraTS19_2013_4_1
Loading slices from: BraTS19_2013_18_1
Loading slices from: BraTS19_2013_23_1
Loading slices from: BraTS19_CBICA_AUA_1
Loading slices from: BraTS19_CBICA_BKV_1
Loading slices from: BraTS19_CBICA_AVT_1
Loading slices from: BraTS19_TCIA02_368_1
Loaded BraTS19_CBICA_ARW_1: (128, 128, 55), (55,)
BraTS19_CBICA_ARW_1 done
BraTS19_TCIA02_208_1 started
Loaded BraTS19_CBICA_ABY_1: (128, 128, 57), (57,)
BraTS19_CBICA_ABY_1 done
BraTS19_CBICA_BIC_1 started
Loaded BraTS19_CBICA_ASU_1: (128, 128, 57), (57,)
BraTS19_CBICA_ASU_1 done
BraTS19_TCIA01_235_1 started
Loaded BraTS19_TCIA03_121_1: (128, 128, 59), (59,)
BraTS19_TCIA03_121_1 done
BraTS19_TCIA02_394_1 started
Loaded BraTS19_TCIA08_162_1: (128, 128, 64), (64,)
BraTS

Loaded BraTS19_TCIA03_265_1: (128, 128, 49), (49,)
BraTS19_TCIA03_265_1 done
BraTS19_CBICA_AUW_1 started
Loaded BraTS19_TCIA01_401_1: (128, 128, 91), (91,)
BraTS19_TCIA01_401_1 done
BraTS19_CBICA_AOS_1 started
Loaded BraTS19_TCIA06_332_1: (128, 128, 78), (78,)
BraTS19_TCIA06_332_1 done
BraTS19_TCIA02_607_1 started
Loaded BraTS19_CBICA_ALU_1: (128, 128, 54), (54,)
BraTS19_CBICA_ALU_1 done
BraTS19_CBICA_ABN_1 started
Loaded BraTS19_TCIA05_444_1: (128, 128, 75), (75,)
BraTS19_TCIA05_444_1 done
BraTS19_TCIA08_280_1 started
Loaded BraTS19_TCIA06_211_1: (128, 128, 76), (76,)
BraTS19_TCIA06_211_1 done
BraTS19_TCIA02_370_1 started
Loaded BraTS19_CBICA_AQT_1: (128, 128, 70), (70,)
BraTS19_CBICA_AQT_1 done
BraTS19_2013_5_1 started
Loaded BraTS19_CBICA_AQQ_1: (128, 128, 62), (62,)
BraTS19_CBICA_AQQ_1 done
BraTS19_CBICA_AAB_1 started
Loaded BraTS19_CBICA_AAG_1: (128, 128, 54), (54,)
BraTS19_CBICA_AAG_1 done
BraTS19_TCIA02_118_1 started
Loaded BraTS19_CBICA_AOD_1: (128, 128, 79), (79,)
BraTS19_CBIC

Loading slices from: BraTS19_CBICA_AME_1
Loading slices from: BraTS19_CBICA_AMH_1
Loaded BraTS19_CBICA_AQY_1: (128, 128, 70), (70,)
BraTS19_CBICA_AQY_1 done
BraTS19_CBICA_BGG_1 started
Loading slices from: BraTS19_CBICA_APY_1
Loading slices from: BraTS19_CBICA_BCF_1
Loading slices from: BraTS19_TCIA01_412_1
Loading slices from: BraTS19_TCIA02_168_1
Loading slices from: BraTS19_TCIA01_335_1
Loading slices from: BraTS19_CBICA_AUX_1
Loading slices from: BraTS19_CBICA_AUQ_1
Loading slices from: BraTS19_CBICA_ANP_1
Loading slices from: BraTS19_CBICA_ANG_1
Loading slices from: BraTS19_CBICA_ASY_1
Loading slices from: BraTS19_CBICA_AYA_1
Loading slices from: BraTS19_TCIA02_455_1
Loading slices from: BraTS19_TCIA08_167_1
Loading slices from: BraTS19_CBICA_ABO_1
Loading slices from: BraTS19_CBICA_APR_1
Loading slices from: BraTS19_CBICA_ASA_1
Loading slices from: BraTS19_TCIA01_429_1
Loading slices from: BraTS19_TCIA04_111_1
Loading slices from: BraTS19_CBICA_AOH_1
Loading slices from: BraTS19_

BraTS19_CBICA_BNR_1 started
Loaded BraTS19_CBICA_ASN_1: (128, 128, 71), (71,)
BraTS19_CBICA_ASN_1 done
Loaded BraTS19_CBICA_BHM_1: (128, 128, 66), (66,)
BraTS19_CBICA_BHM_1 done
Loaded BraTS19_CBICA_ATF_1: (128, 128, 66), (66,)
BraTS19_CBICA_ATF_1 done
Loaded BraTS19_TCIA02_374_1: (128, 128, 72), (72,)
BraTS19_TCIA02_374_1 done
Loaded BraTS19_TCIA04_479_1: (128, 128, 52), (52,)
BraTS19_TCIA04_479_1 done
Loaded BraTS19_CBICA_AQN_1: (128, 128, 46), (46,)
BraTS19_CBICA_AQN_1 done
Loaded BraTS19_TCIA02_430_1: (128, 128, 63), (63,)
BraTS19_TCIA02_430_1 done
Loaded BraTS19_CBICA_BGE_1: (128, 128, 95), (95,)
BraTS19_CBICA_BGE_1 done
Loaded BraTS19_TCIA08_278_1: (128, 128, 55), (55,)
BraTS19_TCIA08_278_1 done
Loaded BraTS19_CBICA_ASE_1: (128, 128, 59), (59,)
BraTS19_CBICA_ASE_1 done
Loading slices from: BraTS19_TCIA04_361_1
Loaded BraTS19_CBICA_ALN_1: (128, 128, 72), (72,)
BraTS19_CBICA_ALN_1 done
Loaded BraTS19_TCIA01_499_1: (128, 128, 44), (44,)
BraTS19_TCIA01_499_1 done
Loaded BraTS19_CBICA

...

Loading slices from: BraTS19_CBICA_AVJ_1
Loading slices from: BraTS19_TCIA02_135_1
Loading slices from: BraTS19_CBICA_AUN_1
Loading slices from: BraTS19_CBICA_ANI_1
Loading slices from: BraTS19_TCIA01_460_1
Loading slices from: BraTS19_TCIA01_150_1
Loading slices from: BraTS19_CBICA_ATV_1
Loading slices from: BraTS19_CBICA_AWH_1
Loading slices from: BraTS19_TCIA06_184_1
Loading slices from: BraTS19_CBICA_AWG_1
Loading slices from: BraTS19_CBICA_AZD_1
Loading slices from: BraTS19_TCIA03_498_1
Loading slices from: BraTS19_CBICA_ASK_1
Loading slices from: BraTS19_CBICA_ANZ_1
Loading slices from: BraTS19_TCIA02_491_1
Loading slices from: BraTS19_2013_21_1
Loading slices from: BraTS19_CBICA_AOO_1
Loaded BraTS19_2013_18_1: (128, 128, 81), (81,)
BraTS19_2013_18_1 done
BraTS19_TCIA02_368_1 started
Loaded BraTS19_CBICA_ARF_1: (128, 128, 67), (67,)
BraTS19_CBICA_ARF_1 done
BraTS19_CBICA_AAL_1 started
Loaded BraTS19_TCIA06_372_1: (128, 128, 76), (76,)
BraTS19_TCIA06_372_1 done
BraTS19_TCIA03_474_

BraTS19_TCIA02_198_1 done
BraTS19_TMC_06290_1 started
Loaded BraTS19_TCIA08_469_1: (128, 128, 79), (79,)
BraTS19_TCIA08_469_1 done
BraTS19_TCIA05_478_1 started
Loaded BraTS19_TCIA02_274_1: (128, 128, 77), (77,)
BraTS19_TCIA02_274_1 done
BraTS19_CBICA_AUR_1 started
Loaded BraTS19_CBICA_AQZ_1: (128, 128, 82), (82,)
BraTS19_CBICA_AQZ_1 done
BraTS19_TCIA01_411_1 started
Loaded BraTS19_TCIA06_247_1: (128, 128, 49), (49,)
BraTS19_TCIA06_247_1 done
BraTS19_CBICA_BDK_1 started
Loaded BraTS19_TCIA02_314_1: (128, 128, 88), (88,)
BraTS19_TCIA02_314_1 done
BraTS19_CBICA_AOD_1 started
Loaded BraTS19_CBICA_AUA_1: (128, 128, 60), (60,)
BraTS19_CBICA_AUA_1 done
BraTS19_CBICA_ALX_1 started
Loaded BraTS19_TCIA03_375_1: (128, 128, 57), (57,)
BraTS19_TCIA03_375_1 done
BraTS19_CBICA_AAG_1 started
Loaded BraTS19_TCIA05_277_1: (128, 128, 93), (93,)
BraTS19_TCIA05_277_1 done
BraTS19_CBICA_AQQ_1 started
Loaded BraTS19_TCIA01_221_1: (128, 128, 68), (68,)
BraTS19_TCIA01_221_1 done
BraTS19_CBICA_AQT_1 started
Loa

BraTS19_CBICA_AUR_1 done
BraTS19_CBICA_AXM_1 started
Loading slices from: BraTS19_TCIA08_113_1
Loading slices from: BraTS19_CBICA_AYU_1
Loading slices from: BraTS19_TCIA03_296_1
Loading slices from: BraTS19_TCIA02_117_1
Loading slices from: BraTS19_TCIA02_377_1
Loading slices from: BraTS19_CBICA_AYW_1
Loading slices from: BraTS19_CBICA_ARZ_1
Loading slices from: BraTS19_2013_27_1
Loading slices from: BraTS19_CBICA_ATX_1
Loading slices from: BraTS19_CBICA_AVV_1
Loading slices from: BraTS19_TCIA02_370_1
Loading slices from: BraTS19_TCIA05_396_1
Loading slices from: BraTS19_CBICA_BFP_1
Loading slices from: BraTS19_TCIA02_321_1
Loading slices from: BraTS19_CBICA_AWI_1
Loading slices from: BraTS19_TCIA02_607_1
Loading slices from: BraTS19_CBICA_BHK_1
Loading slices from: BraTS19_TCIA02_118_1
Loading slices from: BraTS19_TCIA08_280_1
Loading slices from: BraTS19_CBICA_ABN_1
Loading slices from: BraTS19_2013_13_1
Loading slices from: BraTS19_CBICA_AQD_1
Loading slices from: BraTS19_CBICA_AUW_

BraTS19_CBICA_AYA_1 done
BraTS19_2013_3_1 started
Loaded BraTS19_CBICA_AOC_1: (128, 128, 44), (44,)
BraTS19_CBICA_AOC_1 done
BraTS19_CBICA_BAX_1 started
Loaded BraTS19_CBICA_AQR_1: (128, 128, 37), (37,)
BraTS19_CBICA_AQR_1 done
BraTS19_CBICA_ATF_1 started
Loaded BraTS19_TCIA01_335_1: (128, 128, 89), (89,)
BraTS19_TCIA01_335_1 done
BraTS19_CBICA_AQV_1 started
Loaded BraTS19_CBICA_AME_1: (128, 128, 63), (63,)
BraTS19_CBICA_AME_1 done
BraTS19_TCIA02_331_1 started
Loaded BraTS19_TMC_15477_1: (128, 128, 59), (59,)
BraTS19_TMC_15477_1 done
BraTS19_2013_26_1 started
Loaded BraTS19_TCIA01_412_1: (128, 128, 77), (77,)
BraTS19_TCIA01_412_1 done
BraTS19_TCIA02_430_1 started
Loaded BraTS19_CBICA_AUQ_1: (128, 128, 45), (45,)
BraTS19_CBICA_AUQ_1 done
BraTS19_TCIA01_499_1 started
Loaded BraTS19_CBICA_APY_1: (128, 128, 78), (78,)
BraTS19_CBICA_APY_1 done
BraTS19_CBICA_ATB_1 started
Loading slices from: BraTS19_CBICA_AOP_1
Loaded BraTS19_CBICA_AMH_1: (128, 128, 78), (78,)
BraTS19_CBICA_AMH_1 done
BraTS

Loading slices from: BraTS19_TCIA02_471_1
Loaded BraTS19_2013_14_1: (128, 128, 75), (75,)
BraTS19_2013_14_1 done
Loaded BraTS19_CBICA_AQG_1: (128, 128, 84), (84,)
BraTS19_CBICA_AQG_1 done
Loading slices from: BraTS19_TCIA08_436_1
Loading slices from: BraTS19_TCIA02_171_1
Loaded BraTS19_TMC_11964_1: (128, 128, 82), (82,)
Loading slices from: BraTS19_TMC_21360_1
BraTS19_TMC_11964_1 done
Loading slices from: BraTS19_TMC_30014_1
Loading slices from: BraTS19_CBICA_BNR_1
Loaded BraTS19_CBICA_ABB_1: (128, 128, 50), (50,)
BraTS19_CBICA_ABB_1 done
Loading slices from: BraTS19_CBICA_AYC_1
Loaded BraTS19_TCIA04_361_1: (128, 128, 69), (69,)
BraTS19_TCIA04_361_1 done
BraTS19_CBICA_AQP_1 started
Loaded BraTS19_CBICA_ATD_1: (128, 128, 69), (69,)
BraTS19_CBICA_ATD_1 done
BraTS19_CBICA_AWX_1 started
Loaded BraTS19_TCIA02_471_1: (128, 128, 71), (71,)
BraTS19_TCIA02_471_1 done
BraTS19_CBICA_BGN_1 started
Loaded BraTS19_TCIA08_436_1: (128, 128, 62), (62,)
BraTS19_TCIA08_436_1 done
BraTS19_CBICA_AXL_1 star

In [10]:
%%time

side = 128

percentile = 100

sequence_repo = SequenceRepoBrats19("/data/RMN2/datasets/BRATS2019/HGG_Training_data_Brats_2019")

slice_provider = SliceProvider(sequence_repo, full_brain=False)

dataset = GliomiDataset("/data/RMN2/datasets/BRATS2019/HGG_Training_data_Brats_2019/scaled", slice_provider)

for sequence_name in ["t1","t2","t1ce","flair"]:

    X = dataset.load_dataset([sequence_name], side, percentile)

    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-tumor-crop/bratz-{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)

BraTS19_CBICA_AXQ_1 started
BraTS19_CBICA_ATV_1 started
BraTS19_2013_17_1 started
BraTS19_TCIA02_606_1 started
BraTS19_CBICA_AWH_1 started
BraTS19_CBICA_AVJ_1 started
BraTS19_CBICA_AOO_1 started
BraTS19_TCIA08_242_1 started
BraTS19_TCIA06_184_1 started
BraTS19_TCIA01_460_1 started
BraTS19_2013_2_1 started
BraTS19_CBICA_ASF_1 started
BraTS19_CBICA_ASK_1 started
BraTS19_TCIA03_498_1 started
BraTS19_TCIA06_409_1 started
BraTS19_CBICA_ARF_1 started
BraTS19_TCIA04_328_1 started
BraTS19_TCIA01_231_1 started
BraTS19_2013_21_1 started
BraTS19_CBICA_AXO_1 started
BraTS19_TCIA02_300_1 started
BraTS19_CBICA_AUN_1 started
BraTS19_TCIA04_192_1 started
BraTS19_TCIA01_131_1 started
BraTS19_TCIA02_135_1 started
BraTS19_TCIA08_218_1 started
BraTS19_CBICA_ATN_1 started
BraTS19_CBICA_AQO_1 started
BraTS19_TCIA01_150_1 started
BraTS19_TCIA06_372_1 started
BraTS19_TCIA02_491_1 started
BraTS19_TCIA02_309_1 started
BraTS19_CBICA_ANI_1 started
BraTS19_TCIA01_378_1 started
BraTS19_CBICA_AXW_1 started
BraTS19_C

Loading slices from: BraTS19_CBICA_BKV_1
Loading slices from: BraTS19_CBICA_AAL_1
Loading slices from: BraTS19_2013_4_1
Loading slices from: BraTS19_TCIA03_375_1
Loading slices from: BraTS19_TCIA02_314_1
Loading slices from: BraTS19_CBICA_AVT_1
Loading slices from: BraTS19_CBICA_APZ_1
Loading slices from: BraTS19_CBICA_AOZ_1
Loading slices from: BraTS19_TCIA01_390_1
Loaded BraTS19_TCIA03_138_1: (128, 128, 67), (67,)
BraTS19_TCIA03_138_1 done
BraTS19_TCIA02_208_1 started
Loading slices from: BraTS19_2013_11_1
Loaded BraTS19_TCIA01_221_1: (128, 128, 68), (68,)
BraTS19_TCIA01_221_1 done
BraTS19_CBICA_BIC_1 started
Loading slices from: BraTS19_2013_23_1
Loaded BraTS19_TCIA02_274_1: (128, 128, 77), (77,)
BraTS19_TCIA02_274_1 done
BraTS19_TCIA01_235_1 started
Loading slices from: BraTS19_TCIA02_368_1
Loading slices from: BraTS19_TCIA02_208_1
Loaded BraTS19_CBICA_AYI_1: (128, 128, 28), (28,)
BraTS19_CBICA_AYI_1 done
BraTS19_TCIA02_394_1 started
Loaded BraTS19_TCIA02_322_1: (128, 128, 35), (35

Loaded BraTS19_TCIA05_478_1: (128, 128, 69), (69,)
BraTS19_TCIA05_478_1 done
BraTS19_CBICA_BCL_1 started
Loaded BraTS19_CBICA_AQT_1: (128, 128, 70), (70,)
BraTS19_CBICA_AQT_1 done
BraTS19_2013_5_1 started
Loaded BraTS19_CBICA_ASW_1: (128, 128, 76), (76,)
Loaded BraTS19_CBICA_ASO_1: (128, 128, 85), (85,)
BraTS19_CBICA_ASW_1 done
BraTS19_CBICA_BHV_1 started
BraTS19_CBICA_ASO_1 done
BraTS19_TCIA01_425_1 started
Loaded BraTS19_TCIA03_199_1: (128, 128, 81), (81,)
BraTS19_TCIA03_199_1 done
BraTS19_CBICA_ATX_1 started
Loaded BraTS19_TCIA06_603_1: (128, 128, 80), (80,)
BraTS19_TCIA06_603_1 done
BraTS19_CBICA_APK_1 started
Loading slices from: BraTS19_TCIA02_377_1
Loaded BraTS19_CBICA_BGX_1: (128, 128, 73), (73,)
BraTS19_CBICA_BGX_1 done
Loaded BraTS19_CBICA_AUR_1: (128, 128, 97), (97,)
BraTS19_CBICA_BAP_1 started
BraTS19_CBICA_AUR_1 done
BraTS19_CBICA_AXM_1 started
Loaded BraTS19_TMC_06290_1: (128, 128, 77), (77,)
BraTS19_TMC_06290_1 done
BraTS19_CBICA_AVV_1 started
Loaded BraTS19_CBICA_AQA_1:

Loaded BraTS19_TCIA02_321_1: (128, 128, 67), (67,)
BraTS19_TCIA02_321_1 done
BraTS19_CBICA_APR_1 started
Loading slices from: BraTS19_CBICA_ASA_1
Loading slices from: BraTS19_CBICA_ANP_1
Loaded BraTS19_CBICA_AWI_1: (128, 128, 103), (103,)
BraTS19_CBICA_AWI_1 done
BraTS19_CBICA_ANG_1 started
Loading slices from: BraTS19_TCIA01_412_1
Loaded BraTS19_TCIA02_226_1: (128, 128, 58), (58,)
BraTS19_TCIA02_226_1 done
BraTS19_CBICA_BLJ_1 started
Loaded BraTS19_CBICA_ANV_1: (128, 128, 56), (56,)
Loading slices from: BraTS19_CBICA_ASY_1
BraTS19_CBICA_ANV_1 done
BraTS19_CBICA_AOH_1 started
Loading slices from: BraTS19_CBICA_AUQ_1
Loaded BraTS19_TCIA01_180_1: (128, 128, 81), (81,)
BraTS19_TCIA01_180_1 done
BraTS19_CBICA_ABO_1 started
Loaded BraTS19_CBICA_AQY_1: (128, 128, 70), (70,)
BraTS19_CBICA_AQY_1 done
BraTS19_CBICA_BGG_1 started
Loaded BraTS19_TCIA03_296_1: (128, 128, 73), (73,)
BraTS19_TCIA03_296_1 done
BraTS19_TMC_15477_1 started
Loading slices from: BraTS19_CBICA_AYA_1
Loading slices from: B

BraTS19_CBICA_BNR_1 started
Loaded BraTS19_TCIA02_331_1: (128, 128, 84), (84,)
BraTS19_TCIA02_331_1 done
Loaded BraTS19_TCIA01_147_1: (128, 128, 94), (94,)
BraTS19_TCIA01_147_1 done
Loading slices from: BraTS19_TCIA04_361_1
Loading slices from: BraTS19_CBICA_ATD_1
Loaded BraTS19_TCIA02_608_1: (128, 128, 71), (71,)
BraTS19_TCIA02_608_1 done
Loaded BraTS19_TCIA01_499_1: (128, 128, 44), (44,)
BraTS19_TCIA01_499_1 done
Loaded BraTS19_CBICA_AQN_1: (128, 128, 46), (46,)
BraTS19_CBICA_AQN_1 done
Loaded BraTS19_CBICA_AXJ_1: (128, 128, 81), (81,)
BraTS19_CBICA_AXJ_1 done
Loaded BraTS19_CBICA_ATB_1: (128, 128, 71), (71,)
BraTS19_CBICA_ATB_1 done
Loaded BraTS19_TCIA02_473_1: (128, 128, 50), (50,)
BraTS19_TCIA02_473_1 done
Loaded BraTS19_CBICA_AXN_1: (128, 128, 81), (81,)
BraTS19_CBICA_AXN_1 done
Loaded BraTS19_TCIA01_186_1: (128, 128, 51), (51,)
Loaded BraTS19_TCIA08_234_1: (128, 128, 68), (68,)
BraTS19_TCIA08_234_1 done
BraTS19_TCIA01_186_1 done
Loaded BraTS19_CBICA_ATF_1: (128, 128, 66), (66,)


...

Loading slices from: BraTS19_CBICA_AVJ_1
Loading slices from: BraTS19_2013_25_1
Loading slices from: BraTS19_2013_17_1
Loading slices from: BraTS19_TCIA02_606_1
Loading slices from: BraTS19_CBICA_AQU_1
Loading slices from: BraTS19_TCIA04_192_1
Loading slices from: BraTS19_CBICA_ATN_1
Loading slices from: BraTS19_CBICA_ANZ_1
Loading slices from: BraTS19_TCIA01_460_1
Loading slices from: BraTS19_CBICA_AXO_1
Loaded BraTS19_CBICA_AZD_1: (128, 128, 44), (44,)
BraTS19_CBICA_AZD_1 done
BraTS19_TCIA06_247_1 started
Loaded BraTS19_CBICA_AWH_1: (128, 128, 94), (94,)
BraTS19_CBICA_AWH_1 done
BraTS19_CBICA_BAN_1 started
Loaded BraTS19_TCIA01_378_1: (128, 128, 71), (71,)
BraTS19_TCIA01_378_1 done
BraTS19_CBICA_AUA_1 started
Loaded BraTS19_TCIA08_218_1: (128, 128, 55), (55,)
BraTS19_TCIA08_218_1 done
BraTS19_CBICA_BBG_1 started
Loaded BraTS19_TCIA01_150_1: (128, 128, 54), (54,)
BraTS19_TCIA01_150_1 done
BraTS19_CBICA_ABY_1 started
Loaded BraTS19_TCIA06_409_1: (128, 128, 71), (71,)
BraTS19_TCIA06_409

BraTS19_TCIA06_165_1 done
BraTS19_TCIA01_411_1 started
Loaded BraTS19_CBICA_APZ_1: (128, 128, 75), (75,)
BraTS19_CBICA_APZ_1 done
BraTS19_CBICA_BDK_1 started
Loaded BraTS19_CBICA_AUA_1: (128, 128, 60), (60,)
BraTS19_CBICA_AUA_1 done
BraTS19_CBICA_AOD_1 started
Loaded BraTS19_CBICA_ABE_1: (128, 128, 75), (75,)
BraTS19_CBICA_ABE_1 done
BraTS19_CBICA_ALX_1 started
Loaded BraTS19_CBICA_AQZ_1: (128, 128, 82), (82,)
BraTS19_CBICA_AQZ_1 done
BraTS19_CBICA_AAG_1 started
Loaded BraTS19_TCIA08_205_1: (128, 128, 72), (72,)
BraTS19_TCIA08_205_1 done
BraTS19_CBICA_AQQ_1 started
Loaded BraTS19_TCIA03_474_1: (128, 128, 78), (78,)
BraTS19_TCIA03_474_1 done
BraTS19_CBICA_AQT_1 started
Loaded BraTS19_TCIA02_274_1: (128, 128, 77), (77,)
BraTS19_TCIA02_274_1 done
BraTS19_CBICA_AYG_1 started
Loaded BraTS19_TCIA08_105_1: (128, 128, 84), (84,)
BraTS19_TCIA08_105_1 done
BraTS19_TCIA03_199_1 started
Loaded BraTS19_TCIA08_469_1: (128, 128, 79), (79,)
BraTS19_TCIA08_469_1 done
BraTS19_2013_22_1 started
Loading s

Loading slices from: BraTS19_TCIA01_425_1
Loaded BraTS19_TCIA04_343_1: (128, 128, 56), (56,)
BraTS19_TCIA04_343_1 done
BraTS19_CBICA_ASR_1 started
Loading slices from: BraTS19_TCIA01_448_1
Loading slices from: BraTS19_CBICA_BCL_1
Loading slices from: BraTS19_CBICA_BAP_1
Loading slices from: BraTS19_2013_5_1
Loaded BraTS19_TCIA05_444_1: (128, 128, 75), (75,)
BraTS19_TCIA05_444_1 done
Loading slices from: BraTS19_CBICA_ATX_1
BraTS19_TCIA08_280_1 started
Loading slices from: BraTS19_CBICA_AVV_1
Loaded BraTS19_TMC_27374_1: (128, 128, 68), (68,)
BraTS19_TMC_27374_1 done
BraTS19_CBICA_BHK_1 started
Loading slices from: BraTS19_CBICA_AOS_1
Loaded BraTS19_TCIA06_211_1: (128, 128, 76), (76,)
Loaded BraTS19_CBICA_AVF_1: (128, 128, 68), (68,)
BraTS19_TCIA06_211_1 done
BraTS19_TCIA02_370_1 started
BraTS19_CBICA_AVF_1 done
BraTS19_CBICA_BFP_1 started
Loading slices from: BraTS19_TCIA02_321_1
Loading slices from: BraTS19_CBICA_BGW_1
Loading slices from: BraTS19_CBICA_AXM_1
Loading slices from: BraTS

Loaded BraTS19_CBICA_ANP_1: (128, 128, 55), (55,)
BraTS19_CBICA_ANP_1 done
BraTS19_CBICA_AXN_1 started
Loaded BraTS19_CBICA_AOC_1: (128, 128, 44), (44,)
BraTS19_CBICA_AOC_1 done
BraTS19_CBICA_BAX_1 started
Loaded BraTS19_2013_19_1: (128, 128, 75), (75,)
Loaded BraTS19_CBICA_AUX_1: (128, 128, 65), (65,)
BraTS19_2013_19_1 done
BraTS19_CBICA_BGE_1 started
BraTS19_CBICA_AUX_1 done
BraTS19_CBICA_AXJ_1 started
Loaded BraTS19_TCIA01_335_1: (128, 128, 89), (89,)
BraTS19_TCIA01_335_1 done
BraTS19_CBICA_AQV_1 started
Loaded BraTS19_CBICA_APY_1: (128, 128, 78), (78,)
BraTS19_CBICA_APY_1 done
BraTS19_CBICA_ATB_1 started
Loaded BraTS19_CBICA_BCF_1: (128, 128, 90), (90,)
BraTS19_CBICA_BCF_1 done
BraTS19_TCIA01_190_1 started
Loaded BraTS19_2013_7_1: (128, 128, 65), (65,)
BraTS19_2013_7_1 done
BraTS19_TCIA03_257_1 started
Loaded BraTS19_CBICA_ASA_1: (128, 128, 79), (79,)
BraTS19_CBICA_ASA_1 done
BraTS19_TCIA08_234_1 started
Loaded BraTS19_CBICA_ASY_1: (128, 128, 60), (60,)
BraTS19_CBICA_ASY_1 done
Bra

BraTS19_TCIA08_278_1 done
Loaded BraTS19_CBICA_AQG_1: (128, 128, 84), (84,)
BraTS19_CBICA_AQG_1 done
Loaded BraTS19_CBICA_AOP_1: (128, 128, 60), (60,)
BraTS19_CBICA_AOP_1 done
BraTS19_CBICA_ABB_1 started
Loaded BraTS19_2013_26_1: (128, 128, 35), (35,)
BraTS19_2013_26_1 done
BraTS19_CBICA_AYC_1 started
Loaded BraTS19_CBICA_ASV_1: (128, 128, 80), (80,)
BraTS19_CBICA_ASV_1 done
Loaded BraTS19_TMC_11964_1: (128, 128, 82), (82,)
BraTS19_TMC_11964_1 done
Loaded BraTS19_TCIA04_361_1: (128, 128, 69), (69,)
BraTS19_TCIA04_361_1 done
BraTS19_CBICA_AQP_1 started
Loaded BraTS19_TCIA02_171_1: (128, 128, 53), (53,)
BraTS19_TCIA02_171_1 done
BraTS19_2013_12_1 started
Loaded BraTS19_CBICA_ATD_1: (128, 128, 69), (69,)
BraTS19_CBICA_ATD_1 done
BraTS19_CBICA_AWX_1 started
Loaded BraTS19_TCIA08_436_1: (128, 128, 62), (62,)
BraTS19_TCIA08_436_1 done
BraTS19_CBICA_AXL_1 started
Loaded BraTS19_TCIA02_471_1: (128, 128, 71), (71,)
BraTS19_TCIA02_471_1 done
BraTS19_CBICA_BGN_1 started
Loading slices from: BraTS

# Generation of PASQUINI dataset

In [ ]:
%%time

side = 64

percentile = 100

sequence_repo = SequenceRepoGliomi("/data/RMN/dataset-gliomi")

slice_provider = SliceProvider(sequence_repo, full_brain=True)

dataset = GliomiDataset("/data/RMN/dataset-gliomi/scaled", slice_provider)

for sequence_name in ["T1", "T2", "ADC", "FLAIR", "MPRAGE", "rCBV"]:
    
    X = dataset.load_dataset([sequence_name], side, percentile)
    
    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-full-brain/{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)
    


In [ ]:
%%time

side = 64

percentile = 100

sequence_repo = SequenceRepoGliomi("/data/RMN/dataset-gliomi")

slice_provider = SliceProvider(sequence_repo, full_brain=False)

dataset = GliomiDataset("/data/RMN/dataset-gliomi/scaled", slice_provider)

for sequence_name in ["T1", "T2", "ADC", "FLAIR", "MPRAGE", "rCBV"]:
    
    X = dataset.load_dataset([sequence_name], side, percentile)
    
    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-tumor-crop/{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)
    


In [ ]:
%%time

side = 128

percentile = 100

sequence_repo = SequenceRepoGliomi("/data/RMN/dataset-gliomi")

slice_provider = SliceProvider(sequence_repo, full_brain=True)

dataset = GliomiDataset("/data/RMN/dataset-gliomi/scaled", slice_provider)

for sequence_name in ["T1", "T2", "ADC", "FLAIR", "MPRAGE", "rCBV"]:
    
    X = dataset.load_dataset([sequence_name], side, percentile)
    
    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-full-brain/{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)
    


BATTISTA_DOMENICA started
BOVE_A started
DOBRISAN_DORINA started
BIANCHI_GIOVANNI started
DI_LORENZO_TOMMASO started
CALDARONI_ANNA started
ASSANTO_MARIA started
BERNOLA_TERESA started
COLAFRANCESCO_ROCCO started
CRESCENZI_ARMANDO started
DI_MASSA_SERGIO started
CACACE_PAOLO started
BEVILACQUA_RITA started
DIASPRO_G started
BIAVATI_S started
DI_CARLATONIO_MAURIZIO started
DE_SANTIS_GIORGO started
ANGELONI_GIUSEPPINA started
COLAZZO_LUIGI_GIUSEPPE started
DI_MARCO_L started
BARONTINI_MARIA_GIOVANNA started
CAMPLESE_CANDEROLA started
ALESSANDRINI_GLAUCO started
BAGNOLI_VINCENZO started
CAPEZZONE started
BERTUZZI_LUISA started
D_ANGELI_ANNUNZIATA started
BIANCHI_ORAZIO started
CARZEDDA_PAOLO started
DI_MASO_SIMONE started
BERGNACH_SILVANO started
CARULLI_L started
COSTANZI_P started
DEL_BOVE_PIERINA started
BOEZI_MARIO started
DARIDA started
DE_PAOLI_R started
CATALANI_F started
CIMPUREANU_N started
CAMACCI_FILIBERTO started
Loading slices from: DI_LORENZO_TOMMASO
Loading slices from: BAT

LO_BELLO_MARIO done
PIERINI_CATERINA started
Loaded LONGO_ROSALIA: (128, 128, 65), (65,)
LONGO_ROSALIA done
PINEDA_MARIA_ASSUNTA started
Loaded MARAGNO_CLARA: (128, 128, 55), (55,)
MARAGNO_CLARA done
PISTOIA_CARLO started
Loaded INCITI_DONATA: (128, 128, 68), (68,)
INCITI_DONATA done
PODAGROSI_TERESA started
Loaded BOVE_A: (128, 128, 123), (123,)
BOVE_A done
PODDA_ANTONINO started
Loaded CARULLI_L: (128, 128, 95), (95,)
CARULLI_L done
POMPEI_F started
Loading slices from: MAROCCHI_CORRADO
Loading slices from: MARTINEZ
Loading slices from: MEDICI_GIOVANNA
Loading slices from: NERONE_GIANLUCA
Loading slices from: MUSAT_DORINA
Loading slices from: MONACELLI_LAURA
Loading slices from: MICHELI_MICHELE
Loading slices from: MASCI_ADA
Loading slices from: MITCHELL_CHARLENE_ANN
Loading slices from: MOSCARDINI_GIACINTO
Loading slices from: ORLANDI_PAOLO
Loading slices from: PALMIERI
Loading slices from: PAGANNONE_GIANNI
Loading slices from: PANETTI
Loading slices from: PODAGROSI_TERESA
Loading s

/tf/local/gliomi/dataset.py:211: RuntimeWarning: invalid value encountered in true_divide
  images_centered = (images - u_extended) / s_extended


RE_ATTILIO done
Loaded RUSCITO_ELISABETTA: (128, 128, 38), (38,)
RUSCITO_ELISABETTA done
Loaded RICCI_ALESSANDRO: (128, 128, 49), (49,)
RICCI_ALESSANDRO done
Loaded ROMITO_ORAZIO: (128, 128, 53), (53,)
ROMITO_ORAZIO done
Loaded ROSARI_NANDO: (128, 128, 75), (75,)
ROSARI_NANDO done
Loading slices from: TOMAO_ANGELO
Loading slices from: TOMEO_VINCENZO
Loaded SALA_CLARA: (128, 128, 70), (70,)
SALA_CLARA done
Loaded SALTARELLI_DOMENICO: (128, 128, 75), (75,)
SALTARELLI_DOMENICO done
Loaded RUSSO_IDA: (128, 128, 73), (73,)
RUSSO_IDA done
Loading slices from: TIBERI_GIUSEPPE
Loading slices from: ZANATTA_CARLO
Loaded SOLOVIY_VOLODYMYR: (128, 128, 63), (63,)
SOLOVIY_VOLODYMYR done
Loaded TAVOLUCCI_MARIA_RITA: (128, 128, 76), (76,)
TAVOLUCCI_MARIA_RITA done
Loaded STAN_FLORENTINA: (128, 128, 78), (78,)
STAN_FLORENTINA done
Loaded SYKULA_GRAZYNA_BARBARA: (128, 128, 69), (69,)
SYKULA_GRAZYNA_BARBARA done
Loaded TEMPESTINI_MARISA: (128, 128, 63), (63,)
TEMPESTINI_MARISA done
Loaded STEFANINI_CLORI

DI_MASSA_SERGIO done
MARAGNO_CLARA started
Loading slices from: LANDONE_ANNUNZIATA
Loading slices from: LO_BELLO_MARIO
Loading slices from: LONGO_ROSALIA
Loading slices from: LIOCE_CARMELA
Loading slices from: LABELLA_ADRIANA
Loading slices from: MAIOLINI_SANTA
Loading slices from: LUPO_ASSUNTA
Loading slices from: DE_PAOLI_R
Loading slices from: DI_MARCO_L
Loading slices from: DIASPRO_G
Loading slices from: BIAVATI_S
Loading slices from: BALZAMO_A
Loading slices from: ASTA_G
Loading slices from: BIANCHI_S
Loading slices from: CIMPUREANU_N
Loaded FARELLA_PAOLO: (128, 128, 32), (32,)
FARELLA_PAOLO done
MARCOLINI started
Loaded D_ANGELI_ANNUNZIATA: (128, 128, 101), (101,)
D_ANGELI_ANNUNZIATA done
MARIANI_BERNARDO started
Loaded FLORIO_FRANCESCO_PAOLO: (128, 128, 65), (65,)
FLORIO_FRANCESCO_PAOLO done
MAROCCHI_CORRADO started
Loaded FILIPPONI_QUINTINO: (128, 128, 66), (66,)
FILIPPONI_QUINTINO done
MARTINEZ started
Loaded GEGGI_GIULIO: (128, 128, 73), (73,)
GEGGI_GIULIO done
MASCI_ADA star

...

PINEDA_MARIA_ASSUNTA done
TEMPORIN_PATRIZIA started
Loading slices from: RUSSO_IDA
Loaded PRINCIPI_ANNA_MARIA: (128, 128, 72), (72,)
PRINCIPI_ANNA_MARIA done
TEOFILI_STEFANO started
Loaded PIERI: (128, 128, 52), (52,)
PIERI done
TESTA_SERGIO started
Loading slices from: SALTARELLI_DOMENICO
Loading slices from: SALA_CLARA
Loaded PROIETTI_MARIA: (128, 128, 90), (90,)
PROIETTI_MARIA done
TIBERI_GIUSEPPE started
Loaded PROIETTI_GIOVANNI: (128, 128, 51), (51,)
PROIETTI_GIOVANNI done
TOMAO_ANGELO started
Loading slices from: RUSNAC_NINA
Loaded EMERY_R_C: (128, 128, 84), (84,)
EMERY_R_C done
TOMEO_VINCENZO started
Loaded QUATTROCIOCCHI_EVELINA: (128, 128, 65), (65,)
QUATTROCIOCCHI_EVELINA done
TROSCIA_M started
Loading slices from: SANTINI_ERMANNO
Loaded DI_MARCO_L: (128, 128, 72), (72,)
DI_MARCO_L done
VERONESI_ROCCO started
Loaded RE_ATTILIO: (128, 128, 50), (50,)
RE_ATTILIO done
VESCIA_M_T started
Loading slices from: SOLOVIY_VOLODYMYR
Loaded RICCI_ALESSANDRO: (128, 128, 49), (49,)
RICCI_A

FIUCCI_A started
Loaded COLAFRANCESCO_ROCCO: (128, 128, 97), (97,)
COLAFRANCESCO_ROCCO done
FLORIO_FRANCESCO_PAOLO started
Loading slices from: CARULLI_L
Loading slices from: CELLINI_T
Loading slices from: BOVE_A
Loaded CARZEDDA_PAOLO: (128, 128, 56), (56,)
CARZEDDA_PAOLO done
FRATINI_RITA started
Loaded CAMACCI_FILIBERTO: (128, 128, 77), (77,)
CAMACCI_FILIBERTO done
GATTAMORTA_NATALINA started
Loaded BIANCHI_ORAZIO: (128, 128, 69), (69,)
BIANCHI_ORAZIO done
Loaded BOEZI_MARIO: (128, 128, 78), (78,)
BOEZI_MARIO done
GEGGI_GIULIO started
GENNARI_CRISTIANO started
Loaded COLETTA_MARIA: (128, 128, 64), (64,)
COLETTA_MARIA done
GIACCHERINI_M started
Loaded BERGNACH_SILVANO: (128, 128, 68), (68,)
BERGNACH_SILVANO done
GIANFELICI_LUISA started
Loaded DARIDA: (128, 128, 46), (46,)
Loaded BATTISTA_DOMENICA: (128, 128, 54), (54,)
DARIDA done
GIOIA_COSMO_DAMIANO started
BATTISTA_DOMENICA done
GIORDANO_STEFANIA started
Loaded BEVILACQUA_RITA: (128, 128, 42), (42,)
BEVILACQUA_RITA done
INCITI_DONA

Loaded BALZAMO_A: (128, 128, 145), (145,)
BALZAMO_A done
RE_ATTILIO started
Loaded CIMPUREANU_N: (128, 128, 106), (106,)
CIMPUREANU_N done
RICCI_ALESSANDRO started
Loaded COSTANZI_P: (128, 128, 158), (158,)
COSTANZI_P done
ROMITO_ORAZIO started
Loaded ASTA_G: (128, 128, 48), (48,)
ASTA_G done
RONZO_F started
Loading slices from: PIERINI_CATERINA
Loaded CATALANI_F: (128, 128, 134), (134,)
CATALANI_F done
ROSARI_NANDO started
Loading slices from: PIERI
Loading slices from: PISTOIA_CARLO
Loading slices from: PINEDA_MARIA_ASSUNTA
Loading slices from: PODAGROSI_TERESA
Loading slices from: PODDA_ANTONINO
Loading slices from: MONTI_E
Loading slices from: PAGNOTTA
Loaded NERONE_GIANLUCA: (128, 128, 67), (67,)
NERONE_GIANLUCA done
RUSCITO_ELISABETTA started
Loading slices from: PRINCIPI_ANNA_MARIA
Loaded ORLANDI_PAOLO: (128, 128, 6), (6,)
ORLANDI_PAOLO done
RUSNAC_NINA started
Loading slices from: PROIETTI_GIOVANNI
Loading slices from: MOVIA_A
Loading slices from: PROIETTI_MARIA
Loaded PAGANNON

Loading slices from: BIANCHI_GIOVANNI
Loading slices from: BARONTINI_MARIA_GIOVANNA
Loading slices from: CALDARONI_ANNA
Loading slices from: DEL_BOVE_PIERINA
Loading slices from: CRESCENZI_ARMANDO
Loading slices from: COLAFRANCESCO_ROCCO
Loading slices from: CAMPLESE_CANDEROLA
Loading slices from: BERTUZZI_LUISA
Loading slices from: BAGNOLI_VINCENZO
Loading slices from: ALESSANDRINI_GLAUCO
Loading slices from: CACACE_PAOLO
Loading slices from: CAMACCI_FILIBERTO
Loading slices from: DARIDA
Loading slices from: COLETTA_MARIA
Loading slices from: BEVILACQUA_RITA
Loading slices from: BIANCHI_ORAZIO
Loading slices from: BOEZI_MARIO
Loading slices from: BATTISTA_DOMENICA
Loading slices from: CARZEDDA_PAOLO
Loading slices from: COSIMI_MASSIMO
Loading slices from: BERGNACH_SILVANO
Loaded COLAZZO_LUIGI_GIUSEPPE: (128, 128, 70), (70,)
COLAZZO_LUIGI_GIUSEPPE done
DIASPRO_G started
Loaded ASSANTO_MARIA: (128, 128, 92), (92,)
ASSANTO_MARIA done
DI_CARLATONIO_MAURIZIO started
Loaded ANGELONI_GIUSEPP

LUPI_GIANCARLO done
NERVEGNA_G started


In [ ]:
%%time

side = 128

percentile = 100

sequence_repo = SequenceRepoGliomi("/data/RMN/dataset-gliomi")

slice_provider = SliceProvider(sequence_repo, full_brain=False)

dataset = GliomiDataset("/data/RMN/dataset-gliomi/scaled", slice_provider)

for sequence_name in ["T1", "T2", "ADC", "FLAIR", "MPRAGE", "rCBV"]:
    
    X = dataset.load_dataset([sequence_name], side, percentile)
    
    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-tumor-crop/{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)
    


In [ ]:
from gliomi import *

In [ ]:
def get_slice_indexes(sequence_repo, subject, percentile, interval):
    
    roi = sequence_repo.get_roi(subject)

    ((rmin, rmax), (cmin, cmax), (zmin, zmax)) = get_bounding_box(roi)

    width = rmax - rmin
    height = cmax - cmin
    z_height = zmax - zmin

    resampled_roi = mask_crop_resize(roi, roi, width, height, z_height)

    roi_sizes = get_roi_size(resampled_roi, 2)
    
    indexes = ordered_index_percentile_of_sizes(roi_sizes, percentile)
    
    return np.sort(indexes[:interval])

In [ ]:
def michele_get_index_list(item):
    sequence_repo, subject, percentile, interval = item
    print("Working on:", subject)
    result = get_slice_indexes(sequence_repo, subject, percentile, interval)
    print(subject, ": (", result.shape[0], ")", result)
    return result

In [ ]:
%%time

import glob
import os
import multiprocessing

from multiprocessing import Pool

sequence_repo = SequenceRepoGliomi("/data/RMN/dataset-gliomi")

items = []
for file in glob.glob("/data/RMN/dataset-gliomi/scaled/*"):
    subject = os.path.basename(file)
    items.append([sequence_repo, subject, 0, 10])
    
print("Starting...")
    
processing_pool = Pool(int(multiprocessing.cpu_count() / 2))
res = processing_pool.map(michele_get_index_list, items)
print(res)


In [ ]:
from gliomi import *

sequence_repo = SequenceRepoGliomi("/data/RMN/dataset-gliomi")

slice_provider = SliceProvider(sequence_repo, full_brain=True)

s = load_data_single_channel_for_subject(slice_provider, "ALESSANDRINI_GLAUCO", "FLAIR", 64, 70)

In [ ]:
s.shape

In [ ]:
import numpy as np

a = np.moveaxis(s, 3, 0)

In [ ]:
a.shape

In [ ]:
a = np.expand_dims(a, axis=3)

In [ ]:
a.shape

In [ ]:
plot_dataset_with_label(a, list(range(a.shape[0])))

In [ ]:
import numpy as np

# Dataset Management

In [ ]:
side = 224

percentile = 100

sequence_repo = SequenceRepoGliomi("/data/RMN/dataset-gliomi")

slice_provider = SliceProvider(sequence_repo, full_brain=True)

dataset = GliomiDataset("/data/RMN/dataset-gliomi/scaled", slice_provider)

for sequence_name in ["T1", "T2", "ADC", "FLAIR", "MPRAGE", "rCBV"]:
    
    X = dataset.load_dataset([sequence_name], side, percentile)
    
    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-full-brain/{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)

In [ ]:
side = 224

percentile = 100

sequence_repo = SequenceRepoGliomi("/data/RMN/dataset-gliomi")

slice_provider = SliceProvider(sequence_repo, full_brain=False)

dataset = GliomiDataset("/data/RMN/dataset-gliomi/scaled", slice_provider)

for sequence_name in ["T1", "T2", "ADC", "FLAIR", "MPRAGE", "rCBV"]:
    
    X = dataset.load_dataset([sequence_name], side, percentile)
    
    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-tumor-crop/{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)

In [ ]:
side = 224

percentile = 100

sequence_repo = SequenceRepoBrats19("/data/RMN2/datasets/BRATS2019/HGG_Training_data_Brats_2019")

slice_provider = SliceProvider(sequence_repo, full_brain=True)

dataset = GliomiDataset("/data/RMN2/datasets/BRATS2019/HGG_Training_data_Brats_2019/scaled", slice_provider)

for sequence_name in ["t1","t2","t1ce","flair"]:
    
    X = dataset.load_dataset([sequence_name], side, percentile)
    
    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-full-brain/bratz-{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)

In [ ]:
side = 224

percentile = 100

sequence_repo = SequenceRepoBrats19("/data/RMN2/datasets/BRATS2019/HGG_Training_data_Brats_2019")

slice_provider = SliceProvider(sequence_repo, full_brain=False)

dataset = GliomiDataset("/data/RMN2/datasets/BRATS2019/HGG_Training_data_Brats_2019/scaled", slice_provider)

for sequence_name in ["t1","t2","t1ce","flair"]:
    
    X = dataset.load_dataset([sequence_name], side, percentile)
    
    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-tumor-crop/bratz-{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)

In [ ]:
side = 224

percentile = 100

with open(f"/data/RMN/dataset-gliomi-cnn/datasets-tumor-crop/{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
    pickle.dump(X, file)

In [ ]:
for sequence_name in ["T1", "T2", "ADC", "FLAIR", "MPRAGE", "rCBV"]:
    
    X = dataset.load_dataset([sequence_name], 224, 100, full_brain=True)
    
    sequences = "-".join(sequence_names).lower()
    
    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-full-brain/{sequences}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)

# Generate Datasets

In [ ]:
%%time

root_path = "/data/RMN/dataset-gliomi-cnn"

for dataset in [f"{root_path}/slices-tumor-crop"]:

    for sequence_name in ["T1", "T2", "rCBV", "ADC", "FLAIR", "MPRAGE"]:

        for percentile in [100]:

            print(dataset, sequence_name, percentile)
            
            save_dataset(dataset, [sequence_name], 224, percentile, "/data/RMN/dataset-gliomi-cnn/2-datasets-tumor-crop")        

In [ ]:
%%time

root_path = "/data/RMN/dataset-gliomi-cnn"

for dataset in [f"{root_path}/slices-full-brain"]:

    for sequence_name in ["T1", "T2", "rCBV", "ADC", "FLAIR", "MPRAGE"]:

        for percentile in [100]:

            print(dataset, sequence_name, percentile)
            
            save_dataset(dataset, [sequence_name], 224, percentile, "/data/RMN/dataset-gliomi-cnn/2-datasets-full-brain")        

In [ ]:
X, y = get_dataset_for_classification(f"{root_path}/2-datasets-tumor-crop/t1-224-100-perc.pickle", 
                                      f"{root_path}/dataset-survivor.csv")

In [ ]:
for dataframe in [f"{root_path}/dataset-survivor.csv", 
                  f"{root_path}/dataset-mgmt.csv", 
                  f"{root_path}/dataset-idh.csv", 
                  f"{root_path}/dataset-ki67.csv", 
                  f"{root_path}/dataset-egfr.csv"]:



In [ ]:
%%time

root_path = "/data/RMN/dataset-gliomi-cnn"

for dataframe in [f"{root_path}/dataset-survivor.csv", 
                  f"{root_path}/dataset-mgmt.csv", 
                  f"{root_path}/dataset-idh.csv", 
                  f"{root_path}/dataset-ki67.csv", 
                  f"{root_path}/dataset-egfr.csv"]:

    for dataset in [f"{root_path}/slices-full-brain"]:
        
            for sequence_name in ["T1", "T2", "rCBV", "ADC", "FLAIR", "MPRAGE"]:
                
                for percentile in [100]:
                    
                    print(dataset, dataframe, sequence_name, percentile)
                    save_dataset(dataset, dataframe, [sequence_name], 224, percentile, "/data/RMN/dataset-gliomi-cnn/2-datasets-full-brain")
        